In [3]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from scipy.stats import pearsonr

In [ ]:
!pip install tensorflow


In [5]:
img_size = (128, 128) 

df = pd.read_csv('./dataset/csv/real_person.csv')
df = df[['name', 'sex']]

df.dropna(inplace = True)
df

,name,sex
A00147,JOHN,Male
A00220,ISIAH,Male
A00360,HOWARD,Male
A00367,RAYMOND,Male
A01054,DARNELL,Male
...,...,...
Y25362,ERIC A.,Male
Y25363,JEFFERY D.,Male
Y25364,KIMBERLY D.,Female
Y25365,KARLEE R.,Female


In [7]:
male_df = df[df['sex'] == 'Male']
female_df = df[df['sex'] == 'Female']
male_df = male_df[:3768]
print(len(male_df))
print(len(female_df))

final_df = pd.concat([male_df, female_df])
final_df

3768
3768


,name,sex
A00147,JOHN,Male
A00220,ISIAH,Male
A00360,HOWARD,Male
A00367,RAYMOND,Male
A01054,DARNELL,Male
...,...,...
Y25284,KAITLYN C.,Female
Y25313,AMANDA,Female
Y25331,MELISSA K,Female
Y25364,KIMBERLY D.,Female


In [9]:
def preprocess_image(path):
    if os.path.exists(path):
        img = load_img(path, target_size = img_size)
        img = img_to_array(img)
        return img

front_images = []
labels = []
front_folder = './dataset/front/front'

for id in final_df.index:

    front_path = os.path.join(front_folder, f"{id}.jpg")
    front_img = preprocess_image(front_path)
    
    if final_df['sex'][id] == 'Male':
        label = 0
    else:
        label = 1

    if front_img is not None:
        front_images.append(front_img)
        labels.append(label)


front_images = np.array(front_images)
labels = np.array(labels)
print(labels)

[0 0 0 ... 1 1 1]


In [45]:
#np.save("front_images1.npy", front_images)

In [46]:
front_images = np.load('front_images1.npy')


In [47]:
print(front_images.shape)

(7401, 128, 128, 3)


In [11]:
X_train_front, X_test_front, y_train, y_test = train_test_split(
    front_images, labels, test_size=0.2, random_state=42
)


In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.optimizers import Adam

In [16]:
def feature_extraction(input_shape):
    input_layer = Input(shape = input_shape)
    x = Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(x)
    x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)

    return input_layer, x

input_shape = (128, 128, 3)

front_input, front_features = feature_extraction(input_shape)

x = Dense(128, activation = 'relu')(front_features)
x = Dense(64, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x)

model = Model(inputs = front_input, outputs = output)
model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 65536)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       8,388,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,416,449 (32.11 MB)

 Trainable params: 8,416,449 (32.11 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
history = model.fit(
    X_train_front, y_train, 
    batch_size=32, epochs=20, validation_split=0.2
)
loss, accuracy = model.evaluate(X_test_front, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 71s 346ms/step - accuracy: 0.7547 - loss: 32.8206 - val_accuracy: 0.8641 - val_loss: 0.3486
Epoch 2/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 82s 348ms/step - accuracy: 0.9010 - loss: 0.2728 - val_accuracy: 0.9103 - val_loss: 0.2617
Epoch 3/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 68s 353ms/step - accuracy: 0.9339 - loss: 0.1702 - val_accuracy: 0.9057 - val_loss: 0.2391
Epoch 4/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 80s 344ms/step - accuracy: 0.9175 - loss: 0.2168 - val_accuracy: 0.9408 - val_loss: 0.1832
Epoch 5/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 67s 348ms/step - accuracy: 0.9698 - loss: 0.0887 - val_accuracy: 0.9226 - val_loss: 0.2476
Epoch 6/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 67s 347ms/step - accuracy: 0.9067 - loss: 0.2327 - val_accuracy: 0.9044 - val_loss: 0.2430
Epoch 7/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 68s 353ms/step - accuracy: 0.9548 - loss: 0.1385 - val_accuracy: 0.8401 - val_loss: 0.3880
Epoch 8/20
193/193 ━━━━━━━━━━━━━━━━━━━━ 66s 344ms/step - accuracy: 0.9270 - loss: 

In [23]:
# Make predictions
model = load_model("./pretrained_models/front_model.h5")
predictions = model.predict(X_test_front)

# Convert probabilities to binary labels
predicted_labels = (predictions > 0.5).astype(int)
y_pred = (predictions > 0.5).astype(int)

y_test = y_test.squeeze()  # Converts a DataFrame with one column into a Series
y_pred = y_pred.squeeze()
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
pearson_corr, _ = pearsonr(y_test, y_pred)
print("Pearson Correlation Coefficient:", pearson_corr)

# Calculate percentage of predictions within the tolerance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display a few predictions
for i in range(len(X_test_front)):
    print(f"True Label: {y_test[i]}, Predicted: {predicted_labels[i][0]}")


47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step
MSE: 0.05941931127616475
R2 Score: 0.7618918859665149
Mean Absolute Error: 0.05941931127616475
Pearson Correlation Coefficient: 0.8809628687035878
Accuracy: 94.06%
True Label: 1, Predicted: 1
True Label: 1, Predicted: 1
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 1, Predicted: 1
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 1, Predicted: 1
True Label: 0, Predicted: 0
True Label: 1, Predicted: 1
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 1, Predicted: 1
True Label: 0, Predicted: 0
True Label: 0, Predicted: 0
True Label: 1, Predicted: 1
True Label: 1, Predicted: 1
True Label: 1, Predicted: 1
True Label: 1, Predicted: 1
True Label: 0, Predicted: 0
True Label: 1, Predicted: 1
True Label: 0, Predicted: 1
True Label: 1, Predicted: 1
True Label: 0, Predicted: 0
True Label: 

In [1]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, color="purple", alpha=0.6)

plt.title('Comparison of Predicted BMI vs Actual BMI')
plt.ylabel('Predicted BMI')
plt.xlabel('Actual BMI')
plt.axis("equal")  # Ensures equal scaling on both axes
plt.grid(alpha=0.3)
plt.show()

NameError: name 'plt' is not defined

In [22]:
model.save('pretrained_models/front_model2.h5')


In [30]:
from tensorflow.keras.models import load_model
# Function to load, preprocess, and predict gender for a single front image
def predict_gender_with_front_image(front_image_path, model_path):
    img_size = (128, 128)  # Target size for images
    front_img = load_img(front_image_path, target_size=img_size)
    front_img = img_to_array(front_img)
    front_img = np.expand_dims(front_img, axis=0) 

    model = load_model(model_path)
    # Make a prediction
    prediction = model.predict(front_img)
    
    # Interpret the prediction
    if prediction[0][0] > 0.5:
        return "Female"
    else:
        return "Male"

# Example usage:
front_image_path = './test_images/Jenna-ortega.jpg'
predicted_gender = predict_gender_with_front_image(front_image_path, 'pretrained_models/front_model2.h5')

print(f"Predicted Gender: {predicted_gender}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
Predicted Gender: Male


In [65]:
predicted_gender = predict_gender_with_front_image('/kaggle/input/testin/img1.jpg', 'front_model.h5')
print(f"Predicted Gender: {predicted_gender}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Predicted Gender: Male


In [66]:
predicted_gender = predict_gender_with_front_image('/kaggle/input/testin/img2.jpg', 'front_model.h5')
print(f"Predicted Gender: {predicted_gender}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted Gender: Female


In [67]:
predicted_gender = predict_gender_with_front_image('/kaggle/input/testin/img3.jpg', 'front_model.h5')
print(f"Predicted Gender: {predicted_gender}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Predicted Gender: Male


In [68]:
predicted_gender = predict_gender_with_front_image('/kaggle/input/testin/img4.jpg', 'front_model.h5')
print(f"Predicted Gender: {predicted_gender}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Predicted Gender: Male
